In [5]:
import json
import configparser
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

# 設定
config = configparser.ConfigParser()
print(config)
# config.read('/mnt/d/Python/config/config_V20.txt') # パスの指定が必要です
config.read(r"C:\Users\Seiya\ML\config\config_V20.txt")
print(config.sections())
accountID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
params = {
    'count': 3,
    'granularity': 'H1',
}
r = instruments.InstrumentsCandles(instrument='USD_JPY', params=params)
response = api.request(r)
print(json.dumps(response, indent=2))

['oanda']
{
  "instrument": "USD_JPY",
  "granularity": "H1",
  "candles": [
    {
      "complete": true,
      "volume": 333,
      "time": "1577955600.000000000",
      "mid": {
        "o": "108.783",
        "h": "108.842",
        "l": "108.776",
        "c": "108.839"
      }
    },
    {
      "complete": true,
      "volume": 301,
      "time": "1577959200.000000000",
      "mid": {
        "o": "108.836",
        "h": "108.866",
        "l": "108.793",
        "c": "108.807"
      }
    },
    {
      "complete": false,
      "volume": 142,
      "time": "1577962800.000000000",
      "mid": {
        "o": "108.804",
        "h": "108.812",
        "l": "108.776",
        "c": "108.793"
      }
    }
  ]
}


In [6]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime
# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument='USD_JPY'):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        return df
    else:
        return df[df.index < end]
# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    #api_key = '取得したアクセストークンを入れてください'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1/60:
        granularity = 'S5'
    elif minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)

    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')

    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})

    return df, last_timestamp
minute = 60
start = datetime.strptime('2018-08-14 06:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.now()
#end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
df = get_period_data(start, end, minute, instrument='USD_JPY')
print(df)

                        open     high      low    close  amount  complete
time                                                                     
2018-08-14 22:00:00  110.893  110.905  110.764  110.807   945.0      True
2018-08-14 23:00:00  110.810  110.950  110.756  110.887  1056.0      True
2018-08-15 00:00:00  110.885  110.944  110.792  110.932   890.0      True
2018-08-15 01:00:00  110.934  111.113  110.930  111.092   764.0      True
2018-08-15 02:00:00  111.096  111.314  111.092  111.248  1126.0      True
...                      ...      ...      ...      ...     ...       ...
2020-01-01 02:00:00  108.671  108.678  108.578  108.584   365.0      True
2020-01-01 03:00:00  108.586  108.642  108.586  108.622   218.0      True
2020-01-01 04:00:00  108.625  108.642  108.610  108.640   127.0      True
2020-01-01 05:00:00  108.642  108.668  108.634  108.666   192.0      True
2020-01-01 06:00:00  108.664  108.680  108.628  108.628   124.0      True

[8590 rows x 6 columns]


In [7]:
df.to_csv('2018-08-14_06_00_00.csv')